In [63]:
import pymongo
from pymongo import MongoClient,ReturnDocument
from bson import ObjectId
from transformers import pipeline
import re
import json
import scipy
import numpy as np
from nltk import sent_tokenize
import pickle
from bson.binary import Binary

In [ ]:
nlp = pipeline("ner")

In [2]:
def search_through_text(txt,list_search_terms):
    dict_occurrences= {}
    for sentence in txt:
        for country_name in list_search_terms:
            if country_name in sentence:
                if country_name in dict_occurrences:
                    dict_occurrences[dict_countries[country_name]] += 1
                elif country_name not in dict_occurrences:
                    dict_occurrences[dict_countries[country_name]] = 1        
    return dict_occurrences

def split_txt_into_sentences(txt):
    temp_txt = txt.replace('-\n','').replace('\n',' ').lower()
    return sent_tokenize(temp_txt)

In [109]:
client = MongoClient('localhost', 27017)
db = client.db_world_bank
reports_coll = db["reports"]

with open('countries.json') as json_file:
    dict_countries = json.load(json_file)

list_search_terms = [key.lower() for key in dict_countries.keys()]    

coll_knowledge_notes = reports_coll.find({'full_text': {'$exists':True}})                                         
for elem in coll_knowledge_notes:
    object_id = elem["_id"]
    txt = elem['full_text']
    txt = split_txt_into_sentences(txt)
    country_mentions = search_through_text(txt,list_search_terms)
    reports_coll.update_one({'_id':object_id},
                            {'$set': {'country_mentions': country_mentions}}
                            )

In [62]:
import pickle
from bson.binary import Binary
embed  = Binary( pickle.dumps( sentence_embeddings, protocol=2) )
npArray = pickle.loads(embed)
npArray

array([[-0.13941753,  0.39568356,  0.74955016, ...,  0.05756772,
         0.63130397,  0.59260756],
       [-0.56037164, -0.01517188,  0.11742869, ...,  0.01140295,
         1.4424396 , -0.3498124 ],
       [-0.05823234,  0.16593416,  1.3067884 , ...,  0.47052392,
        -0.30400547, -0.49766257],
       ...,
       [-0.5157358 , -0.19514468,  0.47450164, ..., -0.44891167,
         1.0223236 ,  0.17980193],
       [-0.5506063 ,  0.14470986, -0.14311115, ...,  0.23329622,
         0.9878426 ,  0.19337408],
       [-0.35497937,  0.12686574,  0.28984013, ...,  0.01897784,
         1.2822379 , -0.27582422]], dtype=float32)

In [71]:
reports_coll.distinct('collection')

['Journals', 'Knowledge Notes', 'Serial Publications']

In [106]:
client = MongoClient('localhost', 27017)
db = client.db_world_bank
reports_coll = db["reports"]


coll_knowledge_notes = reports_coll.find({'collection':'Serial Publications','abstract': {'$exists':True}},no_cursor_timeout=True)                                         

counter=0
for elem in coll_knowledge_notes:
    print(counter,end='\r')
    counter+=1
    temp_txt = split_txt_into_sentences(elem['abstract'])
    if "abstract_embeddings" not in elem.keys():
        object_id = elem["_id"]
        sentence_embeddings = model.encode(temp_txt)
        reports_coll.update_one({'_id':object_id},
                                {'$set': {'abstract_embeddings': Binary(pickle.dumps(sentence_embeddings, protocol=2))}}
                                )


In [102]:
coll_knowledge_notes = reports_coll.find({'collection':'Knowledge Notes','abstract': {'$exists':True}},no_cursor_timeout=True)                                         
counter = 0
for elem in coll_knowledge_notes[0:1]:
    if elem["abstract_embeddings"]:
        counter+=1
    print(counter)
coll_knowledge_notes.count()

1


KeyError: 'abstract_embeddings'

In [67]:
coll_knowledge_notes = reports_coll.find({'collection':'Knowledge Notes','abstract': {'$exists':True}})                                         

for elem in coll_knowledge_notes[:1]:
    print(pickle.loads(elem["abstract_embeddings"]))

[[-0.13941753  0.39568356  0.74955016 ...  0.05756772  0.63130397
   0.59260756]
 [-0.56037164 -0.01517188  0.11742869 ...  0.01140295  1.4424396
  -0.3498124 ]
 [-0.05823234  0.16593416  1.3067884  ...  0.47052392 -0.30400547
  -0.49766257]
 ...
 [-0.5157358  -0.19514468  0.47450164 ... -0.44891167  1.0223236
   0.17980193]
 [-0.5506063   0.14470986 -0.14311115 ...  0.23329622  0.9878426
   0.19337408]
 [-0.35497937  0.12686574  0.28984013 ...  0.01897784  1.2822379
  -0.27582422]]


In [5]:
temp_txt = split_txt_into_sentences(elem['abstract'])

In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(temp_txt)

In [51]:
query = "stimulating innovation"
query_embedding = model.encode(query)

In [52]:
import scipy
import numpy as np
cos_simils = []
for sentence_embedding in sentence_embeddings:
    cos_simil = scipy.spatial.distance.cosine(query_embedding,sentence_embedding)
    cos_simils.append(cos_simil)
    print(scipy.spatial.distance.cosine(query_embedding,sentence_embedding))
np.mean(cos_simils)

0.9841211643069983
0.8235328048467636
0.6839850544929504
0.7265406548976898
0.5106472671031952
0.7271755635738373
1.0077961520291865
0.59110426902771
1.0208117738366127
0.8537859916687012
0.7079342603683472
0.7489686906337738


0.7822003038988138

In [ ]:
len(sentence_embeddings)

In [ ]:
import nltk 
nltk.download('punkt')


In [ ]:
df= pd.read_csv('list_countries.csv')
dict_country_names = {}
for idx in df.index:
    df.loc[idx,"Other names"] = df.loc[idx,"Other names"].strip().lower() + ', '+df.loc[idx,"Short name"].strip().lower()
    lst_other_names = df.loc[idx,"Other names"].strip().lower().split(', ')
    for name in lst_other_names:
        dict_country_names[name] = df.loc[idx,"Short name"].strip().lower()
dict_country_names

In [ ]:
with open('countries.json', 'w') as outfile:
    json.dump(dict_country_names, outfile)

In [ ]:
from nltk import sent_tokenize
txt2 = txt.replace('-\n','').replace('\n',' ').lower()
test_txt = sent_tokenize(txt2)
test_txt

In [ ]:
import json
with open('countries.json') as json_file:
    dict_countries = json.load(json_file)

search_term = []
for key, value in dict_countries.items():
    search_term.append(value)
    
flat_list = list(set([item.strip().lower() for sublist in search_term for item in sublist]))
flat_list

In [ ]:
import json
with open('countries.json') as json_file:
    dict_countries = json.load(json_file)

dict_countries

In [ ]:
list_search_terms

In [ ]:
with open('countries.json') as json_file:
    dict_countries = json.load(json_file)

list_search_terms = [key.lower() for key in dict_countries.keys()]    
def search_through_text(txt,list_search_terms):
    dict_occurrences= {}
    for sentence in txt:
        for country_name in list_search_terms:
            if country_name in sentence:
                if country_name in dict_occurrences:
                    dict_occurrences[dict_countries[country_name]] += 1
                elif country_name not in dict_occurrences:
                    dict_occurrences[dict_countries[country_name]] = 1
                    
    return dict_occurrences
                
search_through_text(test_txt,list_search_terms)


In [ ]:
import re
txt2 = txt.replace('-\n','').replace('\n',' ').lower()

import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

split_into_sentences(txt)

In [ ]:
interesting_entities = []
text_length = len(txt)
for start_range in range(0,text_length,10000):
    entities = nlp(txt[start_range:start_range+10000].replace('\n',''))
    temp_lst = [entity for entity in entities if entity["entity"]=="I-LOC"]
    print(temp_lst)
    interesting_entities += temp_lst

In [ ]:
interesting_entities = [entity for entity in entities if entity["entity"]=="I-LOC"]
interesting_entities

In [ ]:
entities

In [ ]:
interesting_entities